### View Dataset

In [35]:
import pandas as pd
import webbrowser
import os

# Read the dataset into a data table using Pandas
data_table = pandas.read_csv("housesale.csv")

# Create a web page view of the data for easy viewing
html = data_table[0:100].to_html()

# Save the html to a temporary file
with open("data.html", "w") as f:
    f.write(html)
# Open the web page in our web browser
full_filename = os.path.abspath("data.html")
webbrowser.open("file://{}".format(full_filename))

print(data_table.shape)
print(list(data_table.columns))
data_table.head()


(17030, 15)
['Township', 'PIN', 'Class', 'Address', 'Grantor', 'Grantee', 'Sale Type', 'SaleDate', 'Sale Price', 'Notes', 'Latitude', 'Longitude', 'Year', 'Month', 'Quarter']


,Township,PIN,Class,Address,Grantor,Grantee,Sale Type,SaleDate,Sale Price,Notes,Latitude,Longitude,Year,Month,Quarter
0,BURRITT,10-03-300-005,RURAL IMPROVED,CEMETERY RD,MC DERMOTT STEVEN,KINGSBURY SEAN K,MULTI,9/23/15,"193,000",WARRANTY DEED,42.363380,-89.205514,2015,9,3
1,BURRITT,10-03-300-006,RURAL IMPROVED,9473 CEMETERY RD,NOWICKI JENNIFER,KINGSBURY SEAN K,MULTI,9/23/15,"193,000",WARRANTY DEED,42.363320,-89.223087,2015,9,3
2,BURRITT,10-05-400-007,RURAL LAND,CHRISTIAN RD,MC GLONE ANN L TR,OLIVER SEAN,MULTI,8/4/15,"75,000","TRUSTEE'S DEED, NOT\rEXPOSED TO OPEN MRKT",42.301555,-89.147751,2015,8,3
3,BURRITT,10-08-200-003,RURAL IMPROVED,10621 CEMETERY RD,SULLIVAN JAMES E TRUST,OLIVER SEAN,MULTI,8/4/15,"75,000","TRUSTEE'S DEED, NOT\rEXPOSED TO OPEN MRKT",42.362106,-89.245893,2015,8,3
4,BURRITT,10-08-200-018,RURAL IMPROVED,10650 CEMETERY RD,JOHNSON DEBRA L,COMBS SUSAN,PASMT,11/23/15,"175,000",NaN,42.360278,-89.245793,2015,11,4


### Data exploration and visulization

### Train and test data using Gradient boosting algorithm

In [17]:

from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
from sklearn.externals import joblib

# Load the data set
df = pd.read_csv("housesale.csv")
##checking for missing value
df.isnull().sum()


Township         0
PIN              0
Class            1
Address          0
Grantor          2
Grantee          2
Sale Type        0
SaleDate         0
Sale Price       0
Notes         9432
Latitude       481
Longitude      481
Year             0
Month            0
Quarter          0
dtype: int64

In [18]:

# Remove the fields from the data set that we don't want to include in our model
del df['PIN']
del df['Address']
del df['Grantor']
del df['Grantee']
del df['SaleDate']
del df['Notes']



In [29]:
# Drop the rows with Null value
df.dropna(subset=['Longitude','Latitude','Class' ], inplace=True)


In [34]:
df.isnull().sum()
print(df.shape)
print(type('Longitude'))

(16548, 9)
<class 'str'>


In [32]:
# Replace categorical data with one-hot encoded data
features_df = pd.get_dummies(df, columns=['Township', 'Class', 'Sale Type'])

# Remove the sale price from the feature data
del features_df['Sale Price']

# Create the X and y arrays
X = features_df.as_matrix()
y = df['Sale Price'].as_matrix()

# Split the data set in a training set (70%) and a test set (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Fit regression model
model = ensemble.GradientBoostingRegressor(
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=6,
    min_samples_leaf=9,
    max_features=0.1,
    loss='huber',
    random_state=0
)
model.fit(X_train, y_train)

# Save the trained model to a file so we can use it in other programs
joblib.dump(model, 'trained_house_classifier_model.pkl')

# Find the error rate on the training set
mse = mean_absolute_error(y_train, model.predict(X_train))
print("Training Set Mean Absolute Error: %.4f" % mse)

# Find the error rate on the test set
mse = mean_absolute_error(y_test, model.predict(X_test))
print("Test Set Mean Absolute Error: %.4f" % mse)




ValueError: could not convert string to float: '87,000'